In [1]:
## Download laz tiles ##

In [1]:
from API_utils import *
from dask import delayed, compute
from dask.diagnostics import ProgressBar
import os
import subprocess
import multiprocessing
import json
ncores = multiprocessing.cpu_count()
ncores

16

In [2]:
site = 'TEAK'
productcode = 'DP1.30003.001'
data_path = '/home/jovyan/tmp'
date = '2018-06'
os.makedirs(data_path, exist_ok=True)

In [3]:
t0, laz = generate_laz_download_info(productcode, site, date)

results = []
for f in laz:
    results.append(delayed(download_from_NEON_API)(f, data_path))
    
with ProgressBar():
    computed = compute(*results)

[########################################] | 100% Completed |  1min 50.6s


To create an ept we will use entwine.  First we need to make a ```config.json``` to tell entwine the details of what to do..

In [8]:
# make  config.json for entwine 
config = {'input'  : data_path, 
          'output' : os.path.join(data_path, 'NEON_D17_TEAK_DP1_classified_point_cloud_colorized.ept'),
          'srs'    : 'EPSG:26911',
          'threads': ncores}

with open('config.json', 'w') as dst:
    json.dump(config, dst)

Then we can call entwine to create the ept (and remove the config.json which w e no longer need)

In [6]:
cmd = 'entwine build -c config.json'
result = subprocess.run(cmd, shell=True, capture_output=True)

os.remove('config.json')
result.stderr

b"terminate called after throwing an instance of 'nlohmann::detail::type_error'\n  what():  [json.exception.type_error.305] cannot use operator[] with a string argument with string\nAborted (core dumped)\n"